In [1]:
import sys
!{sys.executable} -m pip install cx_Oracle 
import cx_Oracle as cx
import pandas as pd

You should consider upgrading via the '/Users/ahmadli/PycharmProjects/deploymachinelearning/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
pwd

'/Users/ahmadli/PycharmProjects/CTMP-ThesisProject/pre_LDA'

In [3]:
import os
os.chdir("..")

In [4]:
pwd

'/Users/ahmadli/PycharmProjects/CTMP-ThesisProject'

### Function for getting credentials for DB connection

In [5]:
def get_credentials() -> list:
    c = []
    with open('credentials.txt') as f:
        for line in f.readlines():
            try:
                # fetching username and password
                _, value = line.split(": ")
            except:
                # raises error 
                print('Add your username and password in credentials file')
                exit(0)
            c.append(value.rstrip(" \n"))
    return c

### Connect to DB and fetch DataFrame

In [23]:
"-*- Disconnect from VPN -*-"
dsnStr = cx.makedsn("tirpitz.ms.mff.cuni.cz", 1511, "jedenact")
print(dsnStr)
db_credentials = get_credentials()

try: 
    # Connect to DB
    db = cx.connect(*db_credentials, dsn=dsnStr)
    cur = db.cursor() 
    
    #1st Execute Query - get Actors table
    #cur.execute("select ACTORS from IMDB WHERE ROWNUM <= 50")                    
    #db.commit()
    #df = pd.DataFrame(cur.fetchall())
    #df.columns = ["Actors"]
    
    #2st Execute Query - get Ratings table
    cur.execute("select USERID, MOVIEID, RATING from A_MRATINGS WHERE ROWNUM <=10000")         #-- 2nd Query -- 
    db.commit()
    df = pd.DataFrame(cur.fetchall())
    df.columns = ["USERID", "MOVIEID", "RATING"]
    
    # Another way to Query
    #for row in cur.execute("select ACTORS from IMDB"):
    #    print(row)

    print("Table Created successful") 

except cx.DatabaseError as e: 
    print("ERROR", e, "| or check VPN connection!")

else:
    # Close all when done
    if cur: cur.close() 
    if db: db.close() 

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=tirpitz.ms.mff.cuni.cz)(PORT=1511))(CONNECT_DATA=(SID=jedenact)))
Table Created successful


In [ ]:
#df["Actors"]   #if 1st Query

df["RATING"] = df['RATING'].apply(lambda x: 1 if x>=4 else 0)
df.to_pickle('/Users/ahmadli/PycharmProjects/CTMP-ThesisProject/pre_LDA/ratings.pkl')